# Introduction to Network Programming
---

## Identify a Process on Internet
A process on the Internet is identified by (*host*, *port*)
어떤 프로세스냐는 프로세스 id로 지칭 가능, id는 os 마다 다르다. -> 그래서 port 번호를 사용한다.
- Host(machine) by 32-bit IP address (32비트 binary 주소)
  - dotted decimal notation: `'203.253.70.32'` (32비트가 어려우니깐 string으로 표현한다. 10진법으로 1바이트=8비트로)
  - domain name: `'mclab.hufs.ac.kr'` (숫자는 어려워서 domain name을 지정하고 ... 뒤에서 앞으로 간다.)
  - `'localhost' == '127.0.0.1'` (자기 자신을 지칭하는 ip 주소로 생각한다., 자기 자신을 표현하는 방법)( 누구나 localhost를 가지고 있다.)
- Process in the host by 16-bit port number
  - well-known port: 0 ~ 1023(1023은 특별하게 사용한다.)(잘 알려진, 지정된 포드번호)
    - reserved by standard protocols
    - entifies the standard service(server process)
  - registered port: 1024 ~ 49151( 나중에는 그 위도 사용하게 되었다.)(10000번대 이하는 서버에서 사용한다.)
  - dynamic port: 49152 ~ 65535 (dynamically assigned for clients)( 다이나믹하게 구분하는 용도로 사용된다. 다이나믹하게 할당되는데 클라이언트가, 낮은숫자 순서대로 할당된다.)
(8888은 주피터 노트북은 8888로 할당된 것 localhost 머신이 8888로 주피터노트북이 지정되어있다. 디ㅗㄹ트로)
## Client-Server Model 
(한쪽은 client, 다른 쪽은 server로 역할을 분리하였다.)
(server는 서비스를 제공, 그래서 자기 클라이언트가 많이 있다. 클라이언트들이 request를 많이 보내면 처리해서 응답메세지를 보내준다. request가 없으면 할 일이 없다.)
Internet에서 대부분의 application이나 application protocol은 client-server model로 구현되어 있다. 이 모델은 역할/기능이 분리되어 있어 구현이 용이하다. (Peer-to-peer model도 있다.)

Client: server에게 서비스를 request한다. (서비스를 제공하지 않는다.)

Server: client들의 request를 받아 response를 돌려 준다.(client보다 프로그래밍하기가 어렵다.)
- always on: 보통 항상 실행되고 있다. - daemon process로 request가 올 지 몰라서 항상 시행되고 있고, 그래서 부팅될 때 loan된다.)
- 고정 IP가 필요 (client가 server가 설치된 host를 idtify할 수 있도록)((서버가 어디있는지 알아야 클라이언트가 request를 보낼 수 있다.)
- 여러 client들의 request들을 동시에 받고 처리할 수 있어야 한다.
(서버는 여러개를 동시에 받고 처리)

Application protocol: 표준 프로토콜(예를 들어, HTTP(ITF에서 표준으로 정해놓았다.))을 사용할 수도 있고, 사설(경우에 따라서, 그 회사 제품에서만 사용할 수 있는 것처럼)로 새로 정의할 수도 있다. 어느쪽이든 다음의 transport protocol 위에서 구현된다.(클라이언트 프로레스 서버 프로세스에서는 주고 받고 한다.)(커널에 있는 걸 접근하려면 system call(라이브러리 콜이 아니다.) os에 전달 될 때 처리하고 되돌려 보내준다.)

## Transport Protocol 
(이거 아래에 있는것들은 SW에 코드가 존재한다.)(transport - 부터는 os로 존재)
- TCP: 신뢰성있는 전송. connection-oriented. Byte stream으로 전달(패킷 경계가 없다.)(일대일로 connection, 논리적인 연결)(줄줄이 전달되면 수신자측에서 잘라서 봄)
- UDP: 신뢰성 없음(전달하는 메시지가 손실 가능). conntectionless. 패킷으로 전달(패킷 경계가 있다.)

> Byte를 전달할 뿐, 안에 있는 내용이 문자인지, binary인지 구분하지 않는다.(바이트 스트림으로 나간다. 바이트열로 상대쪽 어플리케이션으로)

## Socket API
(우리는 주로TCP를 사용하는 소켓을 사용할 예정)
(systemp call의 일종이다.)(인터넷프로토콜 전용 시스템콜)(그래서TCP, UDP 등을 사용할 수 있다.)
- 가장 널리 쓰이는 Communication API(Application Programming Interface)(애플리케이션에서 프로그래밍할 수 있게 API 제공)
- 여러 protocol suite에 대해 generic API 제공
- 보통 application에서 OS 내부에 존재하는 Transport protocol간에 API를 제공
- Application에서 network layer, link layer(wifi 등등 으로 근데 root 권한이 필요하다)의 서비스를 직접 이용할 수도 있다. (Super user 권한 필요)

## Socket API로 통신하려면 문자열(str)이 아니라 byte 열로 통신해야 한다.
(TCP나 UDP를 사용한다.)( 바이트인 이유는 transport가 바이트로 오기때문에)
### `str`
문자열이며 문자는 unicode라는 유일한 code point(값)을 갖는다. Unicode는 전 세계의 모든 문자를 컴퓨터에서 일관되게 표현하고 다룰 수 있도록 설계된 산업 표준이다. (짤릴 수도 있지만 넘어간다.)

### Encoding
Unicode는 UTF-8, UTF-16 등 다양하게 encoding될 수 있다.( 전세계 거의 모든 언어가 식별가능하도록, 일관되게 중복되지 않게)(sorting하기가 편하다)(유니코드도 문자열)

**UTF-8 encoding**: 문자열로 구성된 메시지를 socket API로 통신하려면 UTF-8으로 unicode를 변환해야 한다. 왜냐하면, unicode 문자는 멀티 바이트여서 중간에 잘릴 수 있다. 더 중요한 것은 HTTP에서 보는 바와 같이 많은 표준 application protocol들은 ASCII 영문자로 명령 등의 protocol 메시지를 정의해 왔기 때문이다.(8비트, 바이트로 인코딩하는 방식)(유니코드에서 UTF-8 인코딩, 그리고 반대로도 가능하다)(소스코드도 이런걸로 할 때 좋다. 거의 사실상 stardard이다.) ( 한글로 쓰면 거의 utf-8로 되기 때문에 깨지게 되는 것이다.)

- 방식은 문자에 따라 1~4 byte로 표현된다. 
- 영문자(7-bit ASCII)는 그대로 1 byte로 표현된다.
- 8-bit ASCII(>=128)나 한글 등 다국어 문자는 문자 당 2~4 byte로 변환된다.

### `bytes` 
(파이썬은 바이트 타입)
`bytes`: immutable sequences of bytes (변경 불가능)
  - 문자열(str)이 아니다. 내용이 무엇이냐에 상관없이 8-bit인 byte들의 열이다.
  - str method 거의 전부 적용 가능

In [1]:
a = 'hello, 안녕'
b = a.encode('utf-8')
print(b)
print(b.hex())
print(len(b))  # number of bytes, 몇 바이트냐
print(type(b)) # 백슬레시 \x 표현할 수 있는 문자가 없어서

b'hello, \xec\x95\x88\xeb\x85\x95'
68656c6c6f2c20ec9588eb8595
13
<class 'bytes'>


### `bytearray` 
Mutable counterpart to `bytes` objects

In [2]:
ba = bytearray(a, encoding='utf-8')
# ba = bytearray(b) 이렇게도 가능하다. list처럼 extend한다.
ba.extend(b'korea')
ba[0] += 1
print(ba)

bytearray(b'iello, \xec\x95\x88\xeb\x85\x95korea')


In [3]:
ACK = 2
ptype, seq, checksum = 2, 5, 0x9a8b
#ptype, seq, checksum = ACK, 5, 0x9a8b
header = bytearray([ptype, seq, checksum >> 8, checksum & 0xff])
#이렇게 집어 넣어서 header로 표현한다. 
print(header)

bytearray(b'\x02\x05\x9a\x8b')


## First UDP socket program - a client

In [4]:
import socket # 소켓을 쓰기위해 소켓 모듈 사용

# create an UDP socket (open UDP socket), 번호하나, 소켓 object가 나온다.
#AF_INET address family  인터넷 TCP/IP family 를 쓰겠다. 주소 family 나 프로토콜 family나 똑같다.
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
#6이면 AF_INET6
#IP 프로토콜을 건드리면 address family 부터 달라져야 한다.
#socket.SOCK_DGRAM 데이터방식그램 소켓

# send message to the UDP echo server,sendto는  original soket API에서 나오는거
s.sendto(b'Hello, World!', ('np.hufs.ac.kr', 7))   
#np.hufs.ac.kr 를 ip address로 바꾸는게 이게 DNS 프로토콜
#7번 포트라는 이미 지정된것, echo 프로토콜 서버하고 연결하라 , 이 메세지를 보내라, 보내달라고 요청 
#근데 UDP라서 가는 길에 손실될 수가 있다. 그러면 revfrom가 계속 blocked 된다.
#커널에 전달해 주세요, read, write도 system call 이다.
#상대에게 덩어리로 전달이 된다. 그러면 보낸 덩어리로 받는다.

# Wait for something arrival with buffer size == 2048 bytes
# and receive message and find the sender
message, sender = s.recvfrom(2048)        
# 소켓은 네트워크
#2048바이트 버퍼를 할당해주세요 커널한테, 근데 이걸 메세지가 초과하면 짤린다.
# UDP 소켓에서 받는거 커널속에 도착한게 있으면 받아달라, 그래서 도착할 때까지 기다린다.(blocked) 
#내 pc에 udp가 누구한테는 모르는데 받은 것, 받은게 있으면 나한테 주세요(꺠어나서)
#네트워크를 왔다갔다하기 때문에 slow system call이다.
print(sender) #누가 보냈는지 ip address 프로토콜
print(message) #받은 메세지 프린팅

# close the socket
# 다쓰고 나면 닫아야 한다.
s.close()

('203.253.70.30', 7)
b'Hello, World!'


UDP는 connectionless protocol이므로  보낼(send) 때마다 목적지(to)를 지정한다.

받기 전에는 누가 보냈는지 모른다. 메시지가 도착해서 받을(recv) 때 비로소 누가 보냈는지(from) 알 수 있다.
- 메지지는 패킷 덩어리로 전달된다. 3차례에 걸쳐 메시지를 `sendto` 했으면, 상대방은 처음 `recvfrom`으로 첫번 째 message를 받고, 다음 `recvfrom`으로 두번 째 message를 받고, ...
- 다만, 어떤 메시지는 중간에 lost될 수 있다.
- 또한, 메시지가 너무(?) 크다면 짤릴 수 있다. 받아 보니, 일부만 도착할 수도..
(이더넷이면 1500바이트 ip  데이터를 싣을 수 있다. 그러면 이것보다 여러개로 짤린다.
(커널하기 나름이다. 짤릴 수 있다)

> UDP socket을 이용하여 application에서 error없이 loss없이 신뢰성있게 주고 받으려면 UDP를 사용하지 말라.
(그냥 그러면 TCP를 쓰라)

> 그럼 언제??? 특별한 경우... 
(이 lost를 참을 수 있는, 약간의 패킷이 없어도 되는 경우, TCP는 하는 일이 너무 많기 때문에)
> 약간이 손실은 참을 수 있는 multimedia application 등에서. (DNS protocol은 UDP socket으로 구현되어 있다.)

## First TCP socket program - a client
#### Open a TCP socket:
(TCP는 커넥션 오리엔티드 프로토콜로 contect를 먼저해야 한다.

In [5]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # create a TCP socket object
print(s)# 소켓 address 프린팅 #1072 오픈된 파일이면 번호를 받음, 커널과 소통하는 번호 , 이 attrubure를 기억하고 있는 것
print(type(s))

<socket.socket fd=1184, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
<class 'socket.socket'>


#### Connect to the echo server:
- connection이 성공하면, connected socket은 local address와 remote(or foreign) address를 기억한다. (여기서 address는 *socket address*로 (IP address, port number)로 구성된다.
- 상대 쪽, 즉, server 쪽에서도 client의 socket address를 기억하고 있다.

In [6]:
s.connect(('np.hufs.ac.kr', 7))    # connect to echo server, 이거는 TCP7번 포트로 echo 서비스, contect하는 상대를 나타내고 있다.
print(s) #laddr 내 머신 address, localaddress , raddr은 remote address
#내 주소는 host 주소에 달려있는 네트워크 인터페이스 카드마다 부여된다.
#커널에서 소켓 API에서 기억하고 있는 것
#contenection이 laddr과 raddr이 있어야 한다. 하나라도 다르면 안된다.
# 명령 프롬트에 ipconfig 치면 나온다. 랜카드의 서브넷 마스크로 부터 있는거 만약에 wifi까지 있으면 내 주소가 많아진다.

<socket.socket fd=1184, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.10.1.108', 51314), raddr=('203.253.70.30', 7)>


#### Send request message via the connected socket

In [7]:
msg = 'Hello, 대한민국!' #커널이 기억하고 있으니깐 보내는 것
s.send(msg.encode('utf-8'))        # send in bytes or bytearray type, 20바이트 메세지를 보낸것이다. 
#내가 여기 20바이트를 보내주세요 하는 것, 
#TCP 소켓버퍼에 보내달라고 한것 20바이트만 성공적으로 저장한것 버퍼에 용량이 없으면 계속 기다리는 것이고 
#수행되서 돌아오면 성공적으로 커널에게 요청하는 것을 성공한 것
#한번에 상다ㅐ방이 다 안받을 수도있다. 

20

> 주의: 보내는 메시지는 UTF-8 code로 encoding, 받은 메시지는 UNICODE로 decoding
- TCP는 byte-stream protocol이다. 해야 한다. (UTF-8은 multi-byte code를 byte stream으로 *serialize*) 
(UTF-8은 20바이트) (TCP는 여러 바이트를 나눠서 보내면 그걸 나눠서 보낼지 덩어리를 보낼지는 TCP 맘이다.)
(상대방도 현재 커널내에서 도착한 것중에서만 받을 수 있다.)
- Unicode와 같은 multi-byte code를 직접 보내면 위험하다.  
- 인터넷 대다수 protocol의 문법은 원래 ASCII로 사용을 전제로 개발되었고, web에서도 UTF-8가 표준

#### Receive reply message

In [8]:
reply = s.recv(1024)#최대크기 버퍼 사이즈 #커넥션을 보내야 이걸 다 할 수 있다. 안하면 blocked 된다.
print(type(reply))
print(reply.decode('utf-8'))       

<class 'bytes'>
Hello, 대한민국!


#### Close the socket:
- cause to send FIN (request to close the TCP session)
- no more use this socket

In [9]:
s.close() # 파일 오픈하면 클로우즈